# Packages

In [28]:
import numpy as np

from ortools.sat.python import cp_model
from ortools.linear_solver import pywraplp
from ortools.graph.python import linear_sum_assignment

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\OR-Tools'

# Overview

One of the most well-known combinatorial optimization problems is the assignment problem. Here's an example: suppose a group of workers needs to perform a set of tasks, and for each worker and task, there is a cost for assigning the worker to the task. The problem is to assign each worker to at most one task, with no two workers performing the same task, while minimizing the total cost.

You can visualize this problem by the graph below, in which there are four workers and four tasks. The edges represent all possible ways to assign workers to tasks. The labels on the edges are the costs of assigning workers to tasks.

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/assignment/assignment.svg'>
</p>

An assignment corresponds to a subset of the edges, in which each worker has at most one edge leading out, and no two workers have edges leading to the same task. One possible assignment is shown below.

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/assignment/assignment_solution.svg'>
</p>

The total cost of the assignment is `70 + 55 + 95 + 45 = 265`.

# Assignment with Teams of Workers

## CP-SAT solution

In [6]:
# Define data
costs = [
    [90, 76, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 105],
    [45, 110, 95, 115],
    [60, 105, 80, 75],
    [45, 65, 110, 95],
]
num_workers = len(costs)
num_tasks = len(costs[0])

team1 = [0, 2, 4]
team2 = [1, 3, 5]
# Maximum total of tasks for any team
team_max = 2

# Create model
model = cp_model.CpModel()

# Create variables
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker}, {task}]')

# Add constraints
# Each worker is assigned to at most one task
for worker in range(num_workers):
    model.AddAtMostOne(x[worker, task] for task in range(num_tasks))

# Each task is assigned to exactly one worker
for task in range(num_tasks):
    model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

# Each team takes at most two tasks
team1_tasks = []
for worker in team1:
    for task in range(num_tasks):
        team1_tasks.append(x[worker, task])
model.Add(sum(team1_tasks) <= team_max)

team2_tasks = []
for worker in team2:
    for task in range(num_tasks):
        team2_tasks.append(x[worker, task])
model.Add(sum(team2_tasks) <= team_max)

# Create the objective
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
model.Minimize(sum(objective_terms))

# Invoke the solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total cost = {solver.ObjectiveValue()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if solver.BooleanValue(x[worker, task]):
                print(f'Worker {worker} assigned to task {task}. Cost = {costs[worker][task]}')
else:
    print('No solution found.')

Total cost = 250.0

Worker 0 assigned to task 3. Cost = 70
Worker 1 assigned to task 0. Cost = 35
Worker 4 assigned to task 2. Cost = 80
Worker 5 assigned to task 1. Cost = 65


## MIP Solution

In [7]:
# Declare solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create variables
# x[i, j] is an array of 0-1 variables, which will be 1 if worker i is assigned to task j
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = solver.BoolVar(f'x[{worker}, {task}]')

# Add constraints
# Each worker is assigned at most 1 task
for worker in range(num_workers):
    solver.Add(solver.Sum([x[worker, task] for task in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker
for task in range(num_tasks):
    solver.Add(solver.Sum([x[worker, task] for worker in range(num_workers)]) == 1)

# Each team takes at most two tasks
team1_tasks = []
for worker in team1:
    for task in range(num_tasks):
        team1_tasks.append(x[worker, task])
solver.Add(solver.Sum(team1_tasks) <= team_max)

team2_tasks = []
for worker in team2:
    for task in range(num_tasks):
        team2_tasks.append(x[worker, task])
solver.Add(solver.Sum(team2_tasks) <= team_max)

# Create the objective
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
solver.Minimize(solver.Sum(objective_terms))

# Invoke solver
status = solver.Solve()

# Display solutions
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if x[worker, task].solution_value() > 0.5:
                print(f'Worker {worker} assigned to task {task}. Cost = {costs[worker][task]}')
else:
    print('No solution found.')

Total cost = 249.99999999999997

Worker 0 assigned to task 2. Cost = 75
Worker 1 assigned to task 0. Cost = 35
Worker 4 assigned to task 3. Cost = 75
Worker 5 assigned to task 1. Cost = 65


# Assignment with Task Sizes

This section describes an assignment problem in which each task has a **size**, which represents how much time or effort the task requires. The total size of the tasks performed by each worker has a fixed bound.

In [8]:
# Define the data
costs = [
    [90, 76, 75, 70, 50, 74, 12, 68],
    [35, 85, 55, 65, 48, 101, 70, 83],
    [125, 95, 90, 105, 59, 120, 36, 73],
    [45, 110, 95, 115, 104, 83, 37, 71],
    [60, 105, 80, 75, 59, 62, 93, 88],
    [45, 65, 110, 95, 47, 31, 81, 34],
    [38, 51, 107, 41, 69, 99, 115, 48],
    [47, 85, 57, 71, 92, 77, 109, 36],
    [39, 63, 97, 49, 118, 56, 92, 61],
    [47, 101, 71, 60, 88, 109, 52, 90],
]
num_workers = len(costs)
num_tasks = len(costs[0])

task_sizes = [10, 7, 3, 12, 15, 4, 11, 5]
# Maximum total of task sizes for any worker
total_size_max = 15

## CP-SAT solution

In [10]:
# Create the model
model = cp_model.CpModel()

# Create variables
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker}, {task}]')

# Create constraints
# Each worker is assigned to at most one task
for worker in range(num_workers):
    model.Add(sum(task_sizes[task] * x[worker, task] for task in range(num_tasks)) <= total_size_max)

# Each task is assigned to exactly one worker
for task in range(num_tasks):
    model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

# Create objective
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
model.Minimize(sum(objective_terms))

# Invoke solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display solutions
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total costs = {solver.ObjectiveValue()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if solver.BooleanValue(x[worker, task]):
                print(f'Worker {worker} assigned to task {task}. Cost = {costs[worker][task]}')
else:
    print('No solution found.')

Total costs = 326.0

Worker 0 assigned to task 6. Cost = 12
Worker 1 assigned to task 0. Cost = 35
Worker 1 assigned to task 2. Cost = 55
Worker 4 assigned to task 4. Cost = 59
Worker 5 assigned to task 5. Cost = 31
Worker 5 assigned to task 7. Cost = 34
Worker 6 assigned to task 1. Cost = 51
Worker 8 assigned to task 3. Cost = 49


## MIP Solution

In [11]:
# Declare solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create variables
# x[i, j] is an array of 0-1 variables, which will be 1 if worker i is assigned to task j.
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = solver.BoolVar(f'x[{worker},{task}]')

# Add constraints
# The total size of the tasks each worker takes on is at most total_size_max.
for worker in range(num_workers):
    solver.Add(
        solver.Sum([
            task_sizes[task] * x[worker, task] for task in range(num_tasks)
        ]) <= total_size_max)

# Each task is assigned to exactly one worker.
for task in range(num_tasks):
    solver.Add(
        solver.Sum([x[worker, task] for worker in range(num_workers)]) == 1)

# Create objective
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
solver.Minimize(solver.Sum(objective_terms))

# Invoke solver
status = solver.Solve()

# Display results
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if x[worker, task].solution_value() > 0.5:
                print(f'Worker {worker} assigned to task {task}.' +
                      f' Cost: {costs[worker][task]}')
else:
    print('No solution found.')

Total cost = 326.0

Worker 0 assigned to task 6. Cost: 12
Worker 1 assigned to task 0. Cost: 35
Worker 1 assigned to task 2. Cost: 55
Worker 4 assigned to task 4. Cost: 59
Worker 5 assigned to task 5. Cost: 31
Worker 5 assigned to task 7. Cost: 34
Worker 6 assigned to task 1. Cost: 51
Worker 8 assigned to task 3. Cost: 49


# Assignment with Allowed Groups

This section describes an assignment problem in which only certain allowed groups of workers can be assigned to the tasks. In the example there are twelve workers, numbered 0 - 11. The allowed groups are combinations of the following pairs of workers.

In [12]:
group1 =  [[2, 3],       # Subgroups of workers 0 - 3
             [1, 3],
             [1, 2],
             [0, 1],
             [0, 2]]

group2 =  [[6, 7],       # Subgroups of workers 4 - 7
             [5, 7],
             [5, 6],
             [4, 5],
             [4, 7]]

group3 =  [[10, 11],     # Subgroups of workers 8 - 11
             [9, 11],
             [9, 10],
             [8, 10],
             [8, 11]]

An allowed group can be any combination of three pairs of workers, one pair from each of group1, group2, and group3. For example, combining `[2, 3]`, `[6, 7]`, and `[10, 11]` results in the allowed group `[2, 3, 6, 7, 10, 11]`. Since each of the three sets contains five elements, the total number of allowed groups is `5 * 5 * 5 = 125`.

Note that a group of workers can be a solution to the problem if it belongs to any one of the allowed groups. In other words, the feasible set consists of points for which **any one of the constraints is satisfied**. This is an example of a **non-convex** problem. By contrast, the MIP Example, is a **convex problem**: for a point to be feasible, all constraints must be satisfied.

For non-convex problems like this one, the CP-SAT solver is usually faster than a MIP solver. The following sections present solutions to the problem using the CP-SAT solver and the MIP solver, and compare the solution times for the two solvers.

In [13]:
costs = [
    [90, 76, 75, 70, 50, 74],
    [35, 85, 55, 65, 48, 101],
    [125, 95, 90, 105, 59, 120],
    [45, 110, 95, 115, 104, 83],
    [60, 105, 80, 75, 59, 62],
    [45, 65, 110, 95, 47, 31],
    [38, 51, 107, 41, 69, 99],
    [47, 85, 57, 71, 92, 77],
    [39, 63, 97, 49, 118, 56],
    [47, 101, 71, 60, 88, 109],
    [17, 39, 103, 64, 61, 92],
    [101, 45, 83, 59, 92, 27],
]
num_workers = len(costs)
num_tasks = len(costs[0])

## CP-SAT solution

To define the allowed groups of workers for the CP-SAT solver, you create binary arrays that indicate which workers belong to a group. For example, for `group1` (workers 0 - 3), the binary vector `[0, 0, 1, 1]` specifies the group containing workers 2 and 3.

In [14]:
group1 = [
    [0, 0, 1, 1],  # Workers 2, 3
    [0, 1, 0, 1],  # Workers 1, 3
    [0, 1, 1, 0],  # Workers 1, 2
    [1, 1, 0, 0],  # Workers 0, 1
    [1, 0, 1, 0],  # Workers 0, 2
]

group2 = [
    [0, 0, 1, 1],  # Workers 6, 7
    [0, 1, 0, 1],  # Workers 5, 7
    [0, 1, 1, 0],  # Workers 5, 6
    [1, 1, 0, 0],  # Workers 4, 5
    [1, 0, 0, 1],  # Workers 4, 7
]

group3 = [
    [0, 0, 1, 1],  # Workers 10, 11
    [0, 1, 0, 1],  # Workers 9, 11
    [0, 1, 1, 0],  # Workers 9, 10
    [1, 0, 1, 0],  # Workers 8, 10
    [1, 0, 0, 1],  # Workers 8, 11
]

In [21]:
# Model
model = cp_model.CpModel()

# Variables
x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = model.NewBoolVar(f'x[{worker}, {task}]')

# Add constraints
# Each worker is assigned to at most one task
for worker in range(num_workers):
    model.AddAtMostOne(x[worker, task] for task in range(num_tasks))

# Each task is assigned to exactly one worker
for task in range(num_tasks):
    model.AddExactlyOne(x[worker, task] for worker in range(num_workers))

For CP-SAT it is not necessary to create all 125 combinations of these vectors in a loop. The CP-SAT solver provides a method, `AllowedAssignments`, which enables you to specify the constraints for the allowed groups separately for each of the three sets of workers (0 - 3, 4 - 7, and 8 - 11). Here's how it works:

In [19]:
# Create variables for each worker, indicating whether they work on some task.
work = {}
for worker in range(num_workers):
    work[worker] = model.NewBoolVar(f'work[{worker}]')

for worker in range(num_workers):
    for task in range(num_tasks):
        model.Add(work[worker] == sum(
            x[worker, task] for task in range(num_tasks)))

# Define the allowed groups of worders
model.AddAllowedAssignments([work[0], work[1], work[2], work[3]], group1)
model.AddAllowedAssignments([work[4], work[5], work[6], work[7]], group2)
model.AddAllowedAssignments([work[8], work[9], work[10], work[11]], group3)

The variables `work[i]` are 0-1 variables that indicate the work status or each worker. That is, `work[i]` equals 1 if worker i is assigned to a task, and 0 otherwise. The line `solver.Add(solver.AllowedAssignments([work[0], work[1], work[2], work[3]], group1))` defines the constraint that the work status of workers 0 - 3 must match one of the patterns in `group1`. You can see the full details of the code in the next section.

In [22]:
# Create objective
objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
model.Minimize(sum(objective_terms))

# Invoke solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Total cost = {solver.ObjectiveValue()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if solver.BooleanValue(x[worker, task]):
                print(f'Worker {worker} assigned to task {task}.' +
                      f' Cost = {costs[worker][task]}')
else:
    print('No solution found.')

Total cost = 239.0

Worker 0 assigned to task 4. Cost = 50
Worker 1 assigned to task 2. Cost = 55
Worker 5 assigned to task 5. Cost = 31
Worker 6 assigned to task 3. Cost = 41
Worker 10 assigned to task 0. Cost = 17
Worker 11 assigned to task 1. Cost = 45


## MIP solution

In [27]:
"""Solve assignment problem for given group of workers."""
from ortools.linear_solver import pywraplp


def main():
    # Data
    costs = [
        [90, 76, 75, 70, 50, 74],
        [35, 85, 55, 65, 48, 101],
        [125, 95, 90, 105, 59, 120],
        [45, 110, 95, 115, 104, 83],
        [60, 105, 80, 75, 59, 62],
        [45, 65, 110, 95, 47, 31],
        [38, 51, 107, 41, 69, 99],
        [47, 85, 57, 71, 92, 77],
        [39, 63, 97, 49, 118, 56],
        [47, 101, 71, 60, 88, 109],
        [17, 39, 103, 64, 61, 92],
        [101, 45, 83, 59, 92, 27],
    ]
    num_workers = len(costs)
    num_tasks = len(costs[0])

    # Allowed groups of workers:
    group1 = [  # Subgroups of workers 0 - 3
        [2, 3],
        [1, 3],
        [1, 2],
        [0, 1],
        [0, 2],
    ]

    group2 = [  # Subgroups of workers 4 - 7
        [6, 7],
        [5, 7],
        [5, 6],
        [4, 5],
        [4, 7],
    ]

    group3 = [  # Subgroups of workers 8 - 11
        [10, 11],
        [9, 11],
        [9, 10],
        [8, 10],
        [8, 11],
    ]

    # Solver.
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # Variables
    # x[worker, task] is an array of 0-1 variables, which will be 1
    # if the worker is assigned to the task.
    x = {}
    for worker in range(num_workers):
        for task in range(num_tasks):
            x[worker, task] = solver.BoolVar(f'x[{worker},{task}]')

    # Constraints
    # The total size of the tasks each worker takes on is at most total_size_max.
    for worker in range(num_workers):
        solver.Add(
            solver.Sum([x[worker, task] for task in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly one worker.
    for task in range(num_tasks):
        solver.Add(
            solver.Sum([x[worker, task] for worker in range(num_workers)]) == 1)

    # Create variables for each worker, indicating whether they work on some task.
    work = {}
    for worker in range(num_workers):
        work[worker] = solver.BoolVar(f'work[{worker}]')

    for worker in range(num_workers):
        solver.Add(work[worker] == solver.Sum(
            [x[worker, task] for task in range(num_tasks)]))

    # Group1
    constraint_g1 = solver.Constraint(1, 1)
    for i in range(len(group1)):
        # a*b can be transformed into 0 <= a + b - 2*p <= 1 with p in [0,1]
        # p is True if a AND b, False otherwise
        constraint = solver.Constraint(0, 1)
        constraint.SetCoefficient(work[group1[i][0]], 1)
        constraint.SetCoefficient(work[group1[i][1]], 1)
        p = solver.BoolVar(f'g1_p{i}')
        constraint.SetCoefficient(p, -2)

        constraint_g1.SetCoefficient(p, 1)

    # Group2
    constraint_g2 = solver.Constraint(1, 1)
    for i in range(len(group2)):
        # a*b can be transformed into 0 <= a + b - 2*p <= 1 with p in [0,1]
        # p is True if a AND b, False otherwise
        constraint = solver.Constraint(0, 1)
        constraint.SetCoefficient(work[group2[i][0]], 1)
        constraint.SetCoefficient(work[group2[i][1]], 1)
        p = solver.BoolVar(f'g2_p{i}')
        constraint.SetCoefficient(p, -2)

        constraint_g2.SetCoefficient(p, 1)

    # Group3
    constraint_g3 = solver.Constraint(1, 1)
    for i in range(len(group3)):
        # a*b can be transformed into 0 <= a + b - 2*p <= 1 with p in [0,1]
        # p is True if a AND b, False otherwise
        constraint = solver.Constraint(0, 1)
        constraint.SetCoefficient(work[group3[i][0]], 1)
        constraint.SetCoefficient(work[group3[i][1]], 1)
        p = solver.BoolVar(f'g3_p{i}')
        constraint.SetCoefficient(p, -2)

        constraint_g3.SetCoefficient(p, 1)

    # Objective
    objective_terms = []
    for worker in range(num_workers):
        for task in range(num_tasks):
            objective_terms.append(costs[worker][task] * x[worker, task])
    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()

    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Total cost = {solver.Objective().Value()}\n')
        for worker in range(num_workers):
            for task in range(num_tasks):
                if x[worker, task].solution_value() > 0.5:
                    print(f'Worker {worker} assigned to task {task}.' +
                          f' Cost: {costs[worker][task]}')
    else:
        print('No solution found.')


if __name__ == '__main__':
    main()

Total cost = 239.00000000000003

Worker 0 assigned to task 4. Cost: 50
Worker 1 assigned to task 2. Cost: 55
Worker 5 assigned to task 5. Cost: 31
Worker 6 assigned to task 3. Cost: 41
Worker 10 assigned to task 0. Cost: 17
Worker 11 assigned to task 1. Cost: 45


# Linear Sum Assignment Solver

This section describes the linear sum assignment solver, a specialized solver for the simple assignment problem, which can be faster than either the MIP or CP-SAT solver. However, the MIP and CP-SAT solvers can handle a much wider array of problems, so in most cases they are the best option.

## Solving the problem

In [31]:
costs = np.array([
    [90, 76, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 105],
    [45, 110, 95, 115]
])

# Let's transform this into 3 parallel vectors (start_nodes, end_nodes, arc_costs)
end_nodes_unraveled, start_nodes_unraveled = np.meshgrid(
    np.arange(costs.shape[1]), np.arange(costs.shape[0])
)
start_nodes = start_nodes_unraveled.ravel()
end_nodes = end_nodes_unraveled.ravel()
arc_costs = costs.ravel()

start_nodes, end_nodes, arc_costs

(array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]),
 array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]),
 array([ 90,  76,  75,  70,  35,  85,  55,  65, 125,  95,  90, 105,  45,
        110,  95, 115]))

The array is the cost matrix, whose i, j entry is the cost for worker i to perform task j. There are four workers, corresponding to the rows of the matrix, and four tasks, corresponding to the columns.

In [32]:
# Create solver
assignment = linear_sum_assignment.SimpleLinearSumAssignment()

# Add constraints
assignment.add_arcs_with_cost(start_nodes, end_nodes, arc_costs)

# Invole solver
status = assignment.solve()

# Display results
if status == assignment.OPTIMAL:
    print(f'Total costs = {assignment.optimal_cost()}\n')
    for i in range(0, assignment.num_nodes()):
        print(f'Worker {i} assined to task {assignment.right_mate(i)}. Cost = {assignment.assignment_cost(i)}')
elif status == assignment.INFEASIBLE:
    print('No assignment is possible.')
elif status == assignment.POSSIBLE_OVERFLOW:
    print('Some input costs are too large and may cause an integer overflow.')

Total costs = 265

Worker 0 assined to task 3. Cost = 70
Worker 1 assined to task 2. Cost = 55
Worker 2 assined to task 1. Cost = 95
Worker 3 assined to task 0. Cost = 45


The following graph shows the solution as the dashed edges in the graph. The numbers next to the dashed edges are their costs. The total wait time of this assignment is the sum of the costs for the dashed edges, which is 265.

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/assignment/linear_assignment.svg'>
</p>

In graph theory, a set of edges in a bipartite graph that matches every node on the left with exactly one node on the right is called a *perfect matching*.

## Solution when workers can't perform all tasks

In the previous example, we assumed that all workers can perform all tasks. But this not always the case - a worker might be unable to perform one or more tasks for various reasons. However, it is easy to modify the program above to handle this.

As an example, suppose that worker 0 is unable to perform task 3. To modify the program to take this into account, make the following changes:
1. Change the 0, 3 entry of the cost matrix to the string `NA`. (Any string will work.)

 ```python   
    cost = [
        [90, 76, 75, 'NA'],
        [35, 85, 55, 65],
        [125, 95, 90, 105],
        [45, 110, 95, 115]
    ]
```

2. In the section of the code that assigns costs to the solver, add the line `if cost[worker][task] != 'NA':`, as shown below. The added line prevents any edge whose entry in the cost matrix is 'NA' from being added to the solver:

```python
    for worker in range(0, rows):
        for task in range(0, cols):
            if cost[worker][task] != 'NA':
                assignment.AddArcWithCost(worker, task, cost[worker][task])
```
After making these changes, you can see the following output:

```text
    Total cost = 276
    Worker 0 assigned to task 1.  Cost = 76
    Worker 1 assigned to task 3.  Cost = 65
    Worker 2 assigned to task 2.  Cost = 90
    Worker 3 assigned to task 0.  Cost = 45
```

Notice that the total cost is higher now than it was for the original problem. This is not surprising, since in the original problem the optimal solution assigned worker 0 to task 3, while in the modified problem that assignment is not allowed.

To see what happens if more workers are unable to perform tasks, you can replace more entries of the cost matrix with `'NA'`, to denote additional workers who can't perform certain tasks:

 ```python   
    cost = [
        [90, 76, 'NA', 'NA'],
        [35, 85, 'NA', 'NA'],
        [125, 95, 'NA', 'NA'],
        [45, 110, 95, 115]
    ]
```

When you run the program this time, you get a negative result:

```text
    No assignment is possible.
```

This means there is no way to assign workers to tasks so that each worker performs a different task. You can see why this is so by looking at the graph for the problem (in which there are no edges corresponding to values of `'NA'` in the cost matrix).

<p align = 'center'>
    <img src = 'https://developers.google.com/static/optimization/images/assignment/linear_assignment_solution.svg'>
</p>

Since the nodes for the three workers 0, 1, and 2 are only connected to the two nodes for tasks 0 and 1, it not possible to assign distinct tasks to these workers.

## The Marriage Theorem

There is a well-known result in graph theory, called [The Marriage Theorem](https://en.wikipedia.org/wiki/Hall%27s_marriage_theorem), which tells us **exactly when you can assign each node on the left to a distinct node on the right, in a bipartite graph** like the one above. Such an assignment is called a *perfect matching*. In a nutshell, the theorem says this is possible if there is **no subset of nodes on the left (like the one in the previous example) whose edges lead to a smaller set of nodes on the right**.

More precisely, the theorem says that a bipartite graph has a perfect matching if and only if for any subset $S$ of the nodes on the left side of the graph, the set of nodes on the right side of the graph that are connected by an edge to a node in $S$ is at least as large as $S$.